#데이터 로드

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import pandas as pd

# CSV 파일 경로 지정
file_path = '/content/drive/MyDrive/우리FISA_MLDL/팀플/bank.csv'

# CSV 파일 읽기
df_bank = pd.read_csv(file_path)

# 데이터 확인
print(df_bank.head())

   age         job  marital  education default  balance housing loan  contact  \
0   59      admin.  married  secondary      no     2343     yes   no  unknown   
1   56      admin.  married  secondary      no       45      no   no  unknown   
2   41  technician  married  secondary      no     1270     yes   no  unknown   
3   55    services  married  secondary      no     2476     yes   no  unknown   
4   54      admin.  married   tertiary      no      184      no   no  unknown   

   day month  duration  campaign  pdays  previous poutcome deposit  
0    5   may      1042         1     -1         0  unknown     yes  
1    5   may      1467         1     -1         0  unknown     yes  
2    5   may      1389         1     -1         0  unknown     yes  
3    5   may       579         1     -1         0  unknown     yes  
4    5   may       673         2     -1         0  unknown     yes  


# EDA

# 모델링(AutoML)

## 기존 방식 ML

In [4]:
# 기존 전처리
from sklearn.preprocessing import StandardScaler
pd.options.display.float_format = '{:.2e}'.format

# bank_df를 복제해 놓고 복제한 테이블에 작업을 시작

df_bank_ready = df_bank.copy()
df_bank_ready = df_bank_ready.drop('duration', axis=1)  # duration은 사후에 기록된 컬럼이기 때문에, 앞으로의 예측에서는 수집할 수 없는 데이터

# 스케일러 객체 만든다
sc = StandardScaler()
num_cols = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']

# 스케일러 객체에 fit 한다
df_bank_ready[num_cols] = sc.fit_transform(df_bank_ready[num_cols])

# 스케일러 객체에 transform 한다

In [5]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# OneHot 인코딩으로 범주형 변수를 숫자로 변환
encoder = OneHotEncoder()
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome'] # 문자열로 된 컬럼들을 0, 1, 2

df_encoded = pd.DataFrame(encoder.fit_transform(df_bank_ready[cat_cols]).toarray())
df_encoded.columns = encoder.get_feature_names_out(cat_cols)

In [6]:
# df_bank_ready에 합쳐줍니다.
df_bank_ready = df_bank_ready.drop(cat_cols, axis=1)
df_bank_ready = pd.concat([df_encoded, df_bank_ready], axis=1)
df_bank_ready.head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,poutcome_other,poutcome_success,poutcome_unknown,age,balance,day,campaign,pdays,previous,deposit
0,1.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,...,0.00e+00,0.00e+00,1.00e+00,1.49e+00,2.53e-01,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,yes
1,1.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,...,0.00e+00,0.00e+00,1.00e+00,1.24e+00,-4.60e-01,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,yes
2,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,1.00e+00,...,0.00e+00,0.00e+00,1.00e+00,-1.95e-02,-8.02e-02,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,yes
3,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,1.00e+00,0.00e+00,0.00e+00,...,0.00e+00,0.00e+00,1.00e+00,1.16e+00,2.94e-01,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,yes
4,1.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,...,0.00e+00,0.00e+00,1.00e+00,1.07e+00,-4.17e-01,-1.27e+00,-1.87e-01,-4.81e-01,-3.63e-01,yes


In [7]:
# deposit  yes =1 , no = 0으로 수치형 변환이 필요합니다.
df_bank_ready['deposit'] = df_bank_ready['deposit'].apply(lambda x: 1 if x =='yes' else 0)
df_bank_ready.head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,poutcome_other,poutcome_success,poutcome_unknown,age,balance,day,campaign,pdays,previous,deposit
0,1.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,...,0.00e+00,0.00e+00,1.00e+00,1.49e+00,2.53e-01,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,1
1,1.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,...,0.00e+00,0.00e+00,1.00e+00,1.24e+00,-4.60e-01,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,1
2,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,1.00e+00,...,0.00e+00,0.00e+00,1.00e+00,-1.95e-02,-8.02e-02,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,1
3,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,1.00e+00,0.00e+00,0.00e+00,...,0.00e+00,0.00e+00,1.00e+00,1.16e+00,2.94e-01,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,1
4,1.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,...,0.00e+00,0.00e+00,1.00e+00,1.07e+00,-4.17e-01,-1.27e+00,-1.87e-01,-4.81e-01,-3.63e-01,1


In [8]:
# 특성 선택
feature = df_bank_ready.drop('deposit', axis=1)

# 타겟 선택
target = df_bank_ready['deposit']

feature.shape, target.shape
# 훈련 데이터와 테스트 데이터 설정
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, shuffle=True, random_state=121)


In [9]:
def evaluate_model(model, x_test, y_test):
    from sklearn import metrics

    # Predict Test Data
    y_pred = model.predict(x_test)

    # Calculate accuracy, precision, recall, f1-score
    acc = metrics.accuracy_score(y_test, y_pred)
    prec = metrics.precision_score(y_test, y_pred)
    rec = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)

    # Calculate area under curve (AUC)
    y_pred_proba = model.predict_proba(x_test)[::,1]
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)

    # Display confussion matrix
    cm = metrics.confusion_matrix(y_test, y_pred)

    return {'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1,
            'fpr': fpr, 'tpr': tpr, 'auc': auc, 'cm': cm}

In [10]:
from sklearn.ensemble import RandomForestClassifier

# Building Random Forest model
# Evaluate Model

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

rf_eval = evaluate_model(rf, X_test, y_test)
rf_eval['auc']

0.7832614728713349

## Auto ML

In [36]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

### 1. 타겟 라벨 제외한것

In [41]:
import pycaret
import pandas as pd
import numpy as np

x = pd.DataFrame(feature, columns=['age', 'balance', 'day', 'campaign', 'pdays', 'previous'])
# x = pd.DataFrame(feature, columns=['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'age', 'balance', 'day', 'campaign', 'pdays', 'previous'])
y = pd.DataFrame(target, columns=['deposit'])

bostonDF = x
bostonDF['deposit'] = y

In [42]:
bostonDF.head()

,age,balance,day,campaign,pdays,previous,deposit
0,1.49e+00,2.53e-01,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,1
1,1.24e+00,-4.60e-01,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,1
2,-1.95e-02,-8.02e-02,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,1
3,1.16e+00,2.94e-01,-1.27e+00,-5.54e-01,-4.81e-01,-3.63e-01,1
4,1.07e+00,-4.17e-01,-1.27e+00,-1.87e-01,-4.81e-01,-3.63e-01,1


In [43]:
pycaret.__version__

from pycaret.classification import *

s = setup(bostonDF, target ='deposit', session_id = 123)

,Description,Value
0,Session id,123
1,Target,deposit
2,Target type,Binary
3,Original data shape,"(11162, 7)"
4,Transformed data shape,"(11162, 7)"
5,Transformed train set shape,"(7813, 7)"
6,Transformed test set shape,"(3349, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


In [44]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.6599,0.7038,0.5130,0.6905,0.5881,0.3092,0.3194,0.5320
lightgbm,Light Gradient Boosting Machine,0.6574,0.7119,0.5303,0.6771,0.5944,0.3054,0.3124,1.1370
gbc,Gradient Boosting Classifier,0.6571,0.7098,0.5103,0.6855,0.5847,0.3034,0.3131,0.6900
xgboost,Extreme Gradient Boosting,0.6471,0.6926,0.5443,0.6528,0.5934,0.2864,0.2901,0.1390
rf,Random Forest Classifier,0.6362,0.6876,0.5638,0.6298,0.5949,0.2667,0.2681,1.1470
et,Extra Trees Classifier,0.6315,0.6675,0.5916,0.6158,0.6033,0.2596,0.2598,0.9790
nb,Naive Bayes,0.6250,0.6564,0.4219,0.6638,0.5158,0.2340,0.2500,0.0630
qda,Quadratic Discriminant Analysis,0.6181,0.6568,0.3909,0.6647,0.4922,0.2180,0.2377,0.1420
lr,Logistic Regression,0.6176,0.6449,0.4227,0.6472,0.5113,0.2195,0.2329,0.6210
ridge,Ridge Classifier,0.6163,0.6431,0.4079,0.6514,0.5016,0.2158,0.2314,0.0400


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [52]:
reg = create_model('lightgbm')
print(reg)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6253,0.6786,0.4784,0.6390,0.5471,0.2387,0.2460
1,0.6675,0.7126,0.5364,0.6934,0.6049,0.3259,0.3339
2,0.6483,0.7012,0.5310,0.6611,0.5889,0.2880,0.2933
3,0.6325,0.6996,0.5270,0.6352,0.5761,0.2567,0.2602
4,0.6735,0.7336,0.5459,0.6990,0.6131,0.3380,0.3457
5,0.6607,0.7136,0.5432,0.6768,0.6027,0.3127,0.3185
6,0.6748,0.7166,0.5297,0.7101,0.6068,0.3394,0.3500
7,0.6620,0.7236,0.5108,0.6949,0.5888,0.3130,0.3237
8,0.6684,0.7211,0.5757,0.6762,0.6219,0.3299,0.3333


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


### 2. 타겟 라벨 제외 안한 것

In [58]:
# 특성 선택
feature_origin = df_bank.drop('deposit', axis=1)

# 타겟 선택
target_origin = df_bank['deposit']

feature_origin.shape, target_origin.shape

((11162, 16), (11162,))

In [75]:
import pycaret
import pandas as pd
import numpy as np

# x = pd.DataFrame(feature_origin, columns=['age', 'balance', 'day', 'campaign', 'pdays', 'previous'])
# x = pd.DataFrame(feature_origin, columns=['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'age', 'balance', 'day', 'campaign', 'pdays', 'previous', 'deposit'])
# y = pd.DataFrame(target_origin, columns=['deposit'])

# bostonDF_origin = x
# bostonDF_origin['deposit'] = y

bostonDF_origin = df_bank

In [72]:
bostonDF_origin.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,age,balance,day,campaign,pdays,previous,deposit
0,admin.,married,secondary,no,yes,no,unknown,may,unknown,59,2343,5,1,-1,0,NaN
1,admin.,married,secondary,no,no,no,unknown,may,unknown,56,45,5,1,-1,0,NaN
2,technician,married,secondary,no,yes,no,unknown,may,unknown,41,1270,5,1,-1,0,NaN
3,services,married,secondary,no,yes,no,unknown,may,unknown,55,2476,5,1,-1,0,NaN
4,admin.,married,tertiary,no,no,no,unknown,may,unknown,54,184,5,2,-1,0,NaN


In [76]:
pycaret.__version__

from pycaret.classification import *

s = setup(bostonDF_origin, target ='deposit', session_id = 123)

,Description,Value
0,Session id,123
1,Target,deposit
2,Target type,Binary
3,Target mapping,"no: 0, yes: 1"
4,Original data shape,"(11162, 17)"
5,Transformed data shape,"(11162, 49)"
6,Transformed train set shape,"(7813, 49)"
7,Transformed test set shape,"(3349, 49)"
8,Numeric features,7
9,Categorical features,9


In [77]:
best_origin = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8591,0.9270,0.8591,0.8611,0.8592,0.7183,0.7198,2.5820
xgboost,Extreme Gradient Boosting,0.8590,0.9246,0.8590,0.8604,0.8590,0.7179,0.7190,0.4900
rf,Random Forest Classifier,0.8517,0.9191,0.8517,0.8539,0.8517,0.7035,0.7053,1.5210
gbc,Gradient Boosting Classifier,0.8469,0.9189,0.8469,0.8480,0.8470,0.6936,0.6944,1.7380
et,Extra Trees Classifier,0.8284,0.9000,0.8284,0.8286,0.8284,0.6559,0.6561,1.5290
ada,Ada Boost Classifier,0.8235,0.9022,0.8235,0.8238,0.8232,0.6453,0.6460,0.9390
lr,Logistic Regression,0.8174,0.8979,0.8174,0.8176,0.8171,0.6331,0.6337,1.3660
lda,Linear Discriminant Analysis,0.8136,0.9000,0.8136,0.8164,0.8126,0.6242,0.6280,0.2990
ridge,Ridge Classifier,0.8135,0.9000,0.8135,0.8163,0.8124,0.6239,0.6277,0.2790
dt,Decision Tree Classifier,0.7886,0.7875,0.7886,0.7890,0.7884,0.5756,0.5762,0.3580


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [78]:
reg = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8670,0.9221,0.8670,0.8686,0.8671,0.7341,0.7352
1,0.8645,0.9333,0.8645,0.8653,0.8645,0.7288,0.7294
2,0.8465,0.9216,0.8465,0.8499,0.8466,0.6937,0.6963
3,0.8579,0.9274,0.8579,0.8598,0.8580,0.7159,0.7174
4,0.8476,0.9171,0.8476,0.8501,0.8477,0.6956,0.6975
5,0.8630,0.9223,0.8630,0.8640,0.8631,0.7259,0.7266
6,0.8745,0.9446,0.8745,0.8748,0.8746,0.7486,0.7488
7,0.8592,0.9321,0.8592,0.8610,0.8592,0.7184,0.7198
8,0.8451,0.9239,0.8451,0.8476,0.8452,0.6905,0.6924


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

# 파라미터 조정

In [79]:
# 하이퍼파라미터 튜닝
tuned_model = tune_model(reg)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8491,0.9157,0.8491,0.8504,0.8492,0.6982,0.6991
1,0.8542,0.9262,0.8542,0.8555,0.8543,0.7084,0.7094
2,0.8581,0.9145,0.8581,0.8603,0.8581,0.7164,0.7181
3,0.8566,0.9229,0.8566,0.8577,0.8567,0.7131,0.7139
4,0.8399,0.9187,0.8399,0.8424,0.8400,0.6803,0.6821
5,0.8566,0.9173,0.8566,0.8575,0.8567,0.7130,0.7136
6,0.8720,0.9386,0.8720,0.8720,0.8720,0.7433,0.7433
7,0.8489,0.9226,0.8489,0.8502,0.8490,0.6978,0.6987
8,0.8464,0.9227,0.8464,0.8487,0.8464,0.6930,0.6948


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [80]:
# 모델 속성 확인   tuned_model.n_features_in_
print("n_features_in_:", getattr(tuned_model, 'n_features_in_', 'Not available'))
print("n_estimators:", getattr(tuned_model, 'n_estimators', 'Not available'))
print("max_depth:", getattr(tuned_model, 'max_depth', 'Not available'))
print("min_samples_split:", getattr(tuned_model, 'min_samples_split', 'Not available'))
print("min_samples_leaf:", getattr(tuned_model, 'min_samples_leaf', 'Not available'))
print("max_features:", getattr(tuned_model, 'max_features', 'Not available'))
print("bootstrap:", getattr(tuned_model, 'bootstrap', 'Not available'))
print("criterion:", getattr(tuned_model, 'criterion', 'Not available'))

n_features_in_: 48
n_estimators: 100
max_depth: -1
min_samples_split: Not available
min_samples_leaf: Not available
max_features: Not available
bootstrap: Not available
criterion: Not available
